In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.stats import mad_std

In [ ]:
np.random.seed(42)

x = np.linspace(0, 100, 1000)

true_std = 1
true_mean = np.pi

y_obs = true_mean + true_std * np.random.randn(len(x))


plt.hist(y_obs, bins=50)
plt.show()

In [ ]:
def lnprior(theta): 
    mean, std = theta
    
    if -10 < mean < 10 and 0 < std < 10: 
        return 0
    return -np.inf

def propose_step(theta, scale): 
    return theta + scale * np.random.randn(len(theta))

def simulate_dataset(theta): 
    mean, std = theta
    return mean + std * np.random.randn(len(x))

def summary_stats(y_sim):

    distance = np.sqrt((y_obs.std() - y_sim.std())**2 / (0.01 * y_obs.std())**2 + 
                       (y_obs.mean() - y_sim.mean())**2 / y_obs.std()**2)
    
    return distance

In [ ]:
from astropy.utils.console import ProgressBar

In [ ]:
# Initial step parameters for the mean and std:
theta = [true_mean, true_std] 

# Number of posterior samples to compute
n_steps = 3000

# `scale` sets the proposal step jump size
scale = 0.1

# `h` is the distance metric threshold for acceptance
h = 1.5

# Some bookkeeping variables:
accepted_steps = 0
samples = np.zeros((n_steps, len(theta)))
printed = set()

while accepted_steps < n_steps: 
    
    # Make a simple progress bar:
    if accepted_steps % 1000 == 0 and accepted_steps not in printed:
        printed.add(accepted_steps)
        print(f'Sample {accepted_steps} of {n_steps}')
    
    # Propose a new step:
    new_theta = propose_step(theta, scale)
    prior = lnprior(new_theta)

    # If proposed step is within prior: 
    if np.isfinite(prior): 

        # Generate a simulated dataset from new parameters
        y_sim = simulate_dataset(new_theta)

        # Compute distance between simulated dataset
        # and the observations
        distance = summary_stats(y_sim)

        # If distance is less than tolerance `h`, accept step:
        if distance <= h: 
            theta = new_theta
            samples[accepted_steps, :] = new_theta
            accepted_steps += 1

In [ ]:
from corner import corner

corner(samples[n_steps//2:, :], truths=[true_mean, true_std], 
       levels=[0.6], labels=['mean', 'std']);